In [5]:
from scipy import signal
import os
import pickle
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
import torchaudio
import torch
import torchaudio.transforms as T
import pyloudnorm as pyln

In [7]:
data_path = "LMTSoundScans"
file_name_list = os.listdir(data_path)
index = np.random.randint(0, len(file_name_list)-1)
target_wav_file = data_path + '/' + file_name_list[index]

data, fs = librosa.load(target_wav_file, sr=44100)
b, a = signal.butter(3, [20 / fs, 1000 / fs], 'bandpass')
data = signal.filtfilt(b, a, data)
print(data.shape)

(211611,)


In [4]:
sd.play(data, 44100)

In [11]:
def bandpass(wav_path):
    data, fs = librosa.load(wav_path, sr=44100)
    b, a = signal.butter(3, [20 / fs, 1000 / fs], 'bandpass')
    data = signal.filtfilt(b, a, data)
    return data

In [17]:
# whole dataset
dataset_path = 'LMTSoundScans'
autoencoder_data = {"sound": [], "texture": [], "filename":[]}

num_record = len(os.listdir(dataset_path))
print(num_record)
n = 0

for sound in os.listdir(dataset_path):
    print(sound)
    wav = bandpass(dataset_path + '/' + sound)
    samples = np.lib.stride_tricks.sliding_window_view(wav,(44100*3))
    samples = samples.squeeze()
    samples = samples[::100,]
    sample_n = samples.shape[0]
    print(samples.shape)
    autoencoder_data['spectrogram'].extend(samples)
    texture_name = sound.split('_')
    texture_name = texture_name[0]
    n += 1
    print(str(n) + '/' + str(num_record) + ', ' + texture_name)
    autoencoder_data['texture'].extend([texture_name] * sample_n)
    autoencoder_data['filename'].extend([sound] * sample_n)

autoencoder_data['spectrogram'] = np.array(autoencoder_data['spectrogram'])
file = open('trainset_LMT_large.pickle', 'wb')
pickle.dump(autoencoder_data, file)
file.close()


1080
G1EpoxyRasterPlate_Sound_Movement_train1.wav
(794, 132300)


KeyError: 'spectrogram'

In [39]:
with open('trainset_7-class.pickle', 'rb') as file:
    autoencoder_data = pickle.load(file)

In [40]:
print(len(autoencoder_data['texture']))

14400
